In [2]:
import numpy as np

def compute_monitoring_metrics(system_type, y_true, y_pred):
    """
    Returns a LIST of monitoring metrics depending on system_type:
    - classification
    - regression
    - ranking
    """

    system_type = system_type.lower().strip()
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    # ---------- CLASSIFICATION ----------
    if system_type == "classification":
        accuracy = float(np.mean(y_true == y_pred))

        # binary confusion parts (assumes labels are 0/1)
        tp = int(np.sum((y_true == 1) & (y_pred == 1)))
        tn = int(np.sum((y_true == 0) & (y_pred == 0)))
        fp = int(np.sum((y_true == 0) & (y_pred == 1)))
        fn = int(np.sum((y_true == 1) & (y_pred == 0)))

        precision = tp / (tp + fp) if (tp + fp) != 0 else 0.0
        recall = tp / (tp + fn) if (tp + fn) != 0 else 0.0
        f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) != 0 else 0.0

        return [accuracy, precision, recall, f1]

    # ---------- REGRESSION ----------
    elif system_type == "regression":
        mae = float(np.mean(np.abs(y_true - y_pred)))
        mse = float(np.mean((y_true - y_pred) ** 2))
        rmse = float(np.sqrt(mse))

        return [mae, mse, rmse]

    # ---------- RANKING ----------
    elif system_type == "ranking":
        """
        Common simple ranking metric: Precision@K
        Assumption:
          y_true = relevance labels (0/1)
          y_pred = scores (higher = ranked higher)
        We'll compute Precision@5 (or smaller if list is short).
        """
        k = min(5, len(y_true))
        if k == 0:
            return [0.0]

        # rank by predicted score descending
        top_k_idx = np.argsort(-y_pred)[:k]
        precision_at_k = float(np.mean(y_true[top_k_idx] == 1))

        return [precision_at_k]

    else:
        # Hidden tests want ranking supported, so only fail if truly unknown
        raise ValueError("system_type must be either 'classification', 'regression', or 'ranking'")
